<center> <img style="float: center;" src="images/CI_horizontal.png" width="450">
<center>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Website</a>
    </span> 

# <center>Data Exploration: Wages</center>
<a href="https://doi.org/10.5281/zenodo.6385522"><img src="https://zenodo.org/badge/DOI/10.5281/zenodo.6385522.svg" alt="DOI"></a>

<center>Benjamin Feder, Sean Simone, Nathan Barrett</center>

## **1. Introduction**
Few would argue against the commonsense notion that individuals and society benefit from a strong education system. Indeed, the median individual employed full time with a bachelor's degree earns about 80% more than a similar individual with a high school diploma.  More educated populations benefit from increased tax revenue, reduced crime rates and dependence on public assistance programs, and greater civic engagement.  It should come as no surprise then that, with the expectation of future benefits, individuals and governments alike invest a tremendous amount of money into education. In New Jersey, state and local expenditures for higher education alone amounted to over \\$7.4 billion in 2017.  Around the same time, 64% of recent New Jersey college graduates had student loan debt averaging about \$34,000.  

There is room for debate, however, about whether the public is fully and fairly realizing the potential benefits of these investments. There is growing concern of a skills gap whereby individuals with postsecondary degrees are unable to fill the available jobs or find consistent employment with the degrees they have. There are also concerns over equity gaps in the workforce outcomes of postsecondary graduates. Accordingly, there is growing interest in developing and using the data systems needed to better understand the education to workforce pipeline. To date, the federal government has allocated over $750 million to states to develop Statewide Longitudinal Data Systems (SLDS) with the goal of making this possible. While not all states have been able to implement such systems, New Jersey has. This notebook will leverage these data to begin to understand the education to workforce pipeline in New Jersey.


## **2. Learning Objectives**

The Applied Data Analytics training uses a project-based approach to develop your analytic skills. You will begin by working with your team to develop and refine a research question. A crucial part of this is data exploration. You will implement techniques using SQL and R to explore and better understand the data that are available to you and refine your research question. This will form the basis of all the other types of analyses you will do in this class and is a crucial first step for any data analysis workflow. As you work through the notebook, we will have checkpoints for you to practice writing code by making small adjustments, but you can also think about how you might apply any of the techniques and code presented with other datasets to address your research question. 

The guiding research questions we will use for the notebooks are quite general: 

>**What are the employment outcomes of the 2012-13 graduating cohort? How do these outcomes vary by cohort characteristics and employer characteristics?** 

This will allow the code we use to have the most versatility. In the last notebook, you defined a cohort of interest. We will now track their earnings and employment outcomes over time. The exploration of the supply side of the labor market will later be supplemented by an analysis of the demand side to enhance our understanding of the overall labor market.

We are going to show just a portion of what you might be interested in investigating to answer these overarching questions, so don't feel restricted by the questions we've decided to try to answer.

#### **Notebook 2 Questions and Goals** 
In this notebook, we focus on seeking answers to the following questions: 
- What are the average quarterly earnings of our cohort? Do they vary by major?
- What are the stable employment outcomes of our cohort? Do they vary by sex?
- What are the most common employment patterns of our cohort?
- We also provide code in the Appendix for you to explore additional employment outcomes.

After completing this notebook you should be able to perform the following analytical tasks:
- Load R libraries and establish a connection to the server
- Link an education cohort to the wage data
- Identify full quarter employment
- Identify wage outcomes by different subgroups

#### **Datasets** ####
We will explore and understand the New Jersey Education to Earnings Data System (NJEEDS) tables in this notebook:
- **Higher Education (OSHE) Completions**: The completions table comes from the Office of the Secretary of Higher Education's (OSHE) Student Unit Record data system (SURE). The data include completions at all levels that are reported to the U.S. Department of Education's Integrated Postsecondary Education Data System (IPEDS) Completions Survey.
- **Unemployment Insurance (UI) wage records**: Data are collected from businesses and industries in New Jersey that participate, as required by law, in the Unemployment Insurance (UI) program and its trust fund. Wages are reported monthly by industry and compiled quarterly. This information is used for administering the UI program and feeds into mandatory reports to the US Department of Labor.

## 3. SQL and R for Processing Data

SQL is designed to allow for quick and efficient processing of massive amounts of information, such as the UI wage records file. Although you may not have trouble narrowing down a cohort from the Completions table in R, you will run into memory issues reading larger tables into R prior to significantly limiting their size. Because we will need to link our original cohort to the UI wage records to begin to understand the cohort's employment outcomes, we have saved our resulting analytical file formed at the end of the first data exploration notebook as a table in SQL. This will allow us to easily perform a linkage to their employment outcomes in SQL, as opposed to reading the entire UI wage records table into R to perform the linkage. The SQL code to form this table is explained in a  [supplemental notebook](Supplemental/Supplemental_Table_Creation_in_SQL.ipynb). Once we have our final table of wage outcomes specific to our cohort within a defined time period, we should be able to read this table into R to perform more complex analyses, as it is just a small subset of the original UI wage records file.
 
We *highly recommend* reviewing this notebook to understand the table creation process in SQL, as it will be necessary for any future work with the UI wage records or other files of a similar size. The final section of the supplemental notebook replicates the data manipulation performed in R required to prepare the original cohort and UI wage records for the linkage in SQL.

> Note: To save an R data frame as a permanent table in SQL, you can use the following code structure:

        qry <- "use database_with_read_write_access;"
        DBI::dbExecute(con, qry)
        DBI::dbWriteTable(conn = con, name = DBI::SQL("dbo.TABLENAME"), value = df_from_R)

## 4. Notebook Setup

Before we can get started, let's load in the necessary R libraries, connect to the server, and load in the table containing our analytical cohort established in the first data exploration notebook. Again, the supplemental notebook contains information about how the transformations performed in R to create our final data frame in the first notebook were applied in SQL to create the permanent table.

In [ ]:
# Database interaction imports
library(odbc, warn.conflicts=F, quietly=T)

# For data manipulation/visualization
library(tidyverse, warn.conflicts=F, quietly=T)

# For faster date conversions
library(lubridate, warn.conflicts=F, quietly=T)

# Use percent() function
library(scales, warn.conflicts=F, quietly=T)

In [ ]:
# Connect to the server
con <- DBI::dbConnect(odbc::odbc(),
                     Driver = "SQL Server",
                     Server = "msssql01.c7bdq4o2yhxo.us-gov-west-1.rds.amazonaws.com",
                     Trusted_Connection = "True")

In [ ]:
# start with cohort created in notebook 1
qry <- "
select * 
from tr_nj_2021.dbo.nb_cohort
"
df_cohort <- dbGetQuery(con, qry)

In [ ]:
# see cohort
head(df_cohort)

## 5. Matching Cohort to Wage Records

Before we can begin to understand the employment outcomes for the cohort established in the first notebook, we need to be able to link the cohort to the wage records. This section will walk you through the entire linkage procedure.

### Understanding the Linkage

Before we can begin the linking process, we need to understand how we can link the two tables. Let's take a look at the `ds_nj_dol.dbo.ui_wages` table and see if we can spot any common variables by which we can create a potential linkage.

In [ ]:
# see ui wage records
qry <- "
select top 5 *
from ds_nj_dol.dbo.ui_wages
where yrq = 20042
"
wage_tbl <- dbGetQuery(con, qry)

wage_tbl

As you can see, the variable `hashed_ssn` is common to each of the two tables. However, we only want to find employment outcomes within a limited analysis time frame.

------

#### **Checkpoint 1: Time Travel**

Given the available variables in `wage_tbl` and `df_cohort`, can you identify potential variables we can use to define a specific time frame (up to three years post-graduation)? Refer to the data dictionaries for complete column definitions.

> Note: You don't need to perform the linkage--we will be doing that in a handful of code cells--but please think about potential variables we might be able to use in the future.

In [ ]:
# which variables? 
# hint: you can use this space to explore the two data frames with glimpse(), head(), names(), or str() for example 
# if you are stuck



--------

### Data Manipulation

There are a few different ways we can approach linking these two tables so that they satisfy a specific time constraint. Although the solution presented may not line up with your answer to Checkpoint 1, it is one that can be applied to a lot of other scenarios.

The general idea is that we will create new variables in each of the tables that represent graduation and job information in terms of calendar dates. From there, we can take advantage of SQL and R's date-specific functions to extract data within a three-year timespan. To create these new variables, we will need to use `new_awarddate` and `awardyearn` from `nb_cohort`, and `yrq` from `ui_wages`.

First, let's create the variable tracking job dates in the wage records table. For consistency, we will convert each `yrq` value (year + quarter of employment) to the first day of the given financial quarter (i.e. Q1: January 1, Q2: April 1, etc.).

#### Wage Records

In [ ]:
# see yrq
wage_tbl %>%
    select(yrq)

To get the month of the first day of the quarter, you can take each quarter, multiply it by 3, and then subtract 2. Once the date is formatted as mm/dd/yyyy, we can use `lubridate`'s `mdy` function to convert the variable to a date.

> Using the `lubridate` package, you could also convert a character type of yyyy/mm/dd to a date with the function `ymd`. Other date formats are supported as well.

In [ ]:
# add date
# the month needs to be converted from a character type to integer to allow for the multiplication and substraction
wage_tbl %>%
    mutate(
        job_date = mdy(paste(as.integer(substring(yrq, 5, 5))*3-2, '01', substring(yrq, 1, 4), sep="/"))
    ) %>%
    select(yrq, job_date)

The SQL-translated version of this updated data frame is available in the table `nb_cohort_wages_dated`. For more information about how the table was created in SQL, refer to the supplemental table creation notebook.

> Note: `nb_cohort_wages_dated` is further restricted to only include wages from individuals in `nb_cohort_dated`. This is done to limit the time it takes to conduct the linkage process.

#### Graduates Cohort

While the data manipulation process for `df_cohort` will follow the same logical approach, it will require one extra step, as there is not an easy trick for getting the graduation months to correspond to the first month within each fiscal quarter. Therefore, we will first need to create a variable corresponding to the first month in each quarter given the graduation month, and then we can use the information at hand to create a rough graduation date. 

In [ ]:
# new month for new_awarddate
df_cohort_mth <- df_cohort %>%
    mutate(
        matched_awd_mth = case_when(
            substring(new_awarddate, 1, 2) %in% c("01", "02", "03") ~ "01",
            substring(new_awarddate, 1, 2) %in% c("04", "05", "06") ~ "04",
            substring(new_awarddate, 1, 2) %in% c("07", "08", "09") ~ "07",
            substring(new_awarddate, 1, 2) %in% c("10", "11", "12") ~ "10",
        )
    )

# see month mapping in practice (now maps to quarters/semesters)
df_cohort_mth %>%
    select(new_awarddate, matched_awd_mth) %>%
    head()

Now that we have coded months, we can create a new variable `grad_date` using an approach similar to our approach for creating `job_date`.

In [ ]:
# add date
df_cohort <- df_cohort_mth %>%
    mutate(
        grad_date = mdy(paste(matched_awd_mth, '01', awardyearn, sep="/"))
    )

df_cohort %>%
    select(grad_date, new_awarddate) %>%
    head()

These two steps are combined using a CTE in SQL and written to the table `nb_cohort_dated`. For more information about how the table was created in SQL, refer to the supplemental table creation notebook.

### Joining Updated Tables

At this point, we have two permanent tables stored in the `tr_nj_2021` database at our disposal to find employment history up to *REDACTED* years after graduation: `nb_cohort_dated` (graduates) and `nb_cohort_dated_wages` (wage records). We can link these two tables based on common `hashed_ssn` values and limit the time frame using SQL's date functions. As a reminder, we would not be able to perform this linkage in R due to memory issues present in storing the wage records as a data frame.

We will use a simple `join` statement and add our time constraints to the `where` clause. The time constraint will be implemented by only taking `job_date` values that occur within REDACTED quarters (REDACTED years plus REDACTED quarter) of graduation. 

The date-specific function we will use in SQL is `dateadd`, as it allows us to add different time intervals to date variables.

> Note: We will filter out all wage records with `wage` values of 0, as employers are only required to report non-zero wages paid during the quarter. In this notebook, we will assume that the records with 0 quarterly wages are reflective of HR efforts to prepare to report future earnings instead of employment with 0 wages earned in the quarter.

In [ ]:
## see linked wages 3 years out
qry <- "
select cd.*, w.hashed_ein, w.wage, w.weeks, w.yrq, w.filedate, w.job_date
from tr_nj_2021.dbo.nb_cohort_dated cd
join tr_nj_2021.dbo.nb_cohort_wages_dated w
on cd.hashed_ssn = w.hashed_ssn
where w.job_date >= cd.grad_date and dateadd(quarter, 13, cd.grad_date) >= w.job_date and wage > 0
"
df_wages <- dbGetQuery(con, qry)

head(df_wages)

-------

#### **Checkpoint 2: Time-Keeping**

Adjust the query above to only include wage records in the two years (8 quarters) prior to graduation. Return five rows to confirm your results. (Hint: you can re-use the dateadd function by switching the order of the variables and reversing the inequality signs.)

In [ ]:
# adjust time interval
# replace ___ with proper where clause
qry <- "
select top 5 cd.*, w.hashed_ein, w.wage, w.weeks, w.yrq, w.filedate, w.job_date
from tr_nj_2021.dbo.nb_cohort_dated cd
join tr_nj_2021.dbo.nb_cohort_wages_dated w
on cd.hashed_ssn = w.hashed_ssn
where ___
"
dbGetQuery(con, qry)

--------

## 6. Quick Exploration

Before we start evaluating different employment measures, we should get a better grasp of the data frame. Let's start by finding the number of individuals, as well as jobs that were linked across the two tables.

In [ ]:
# see number of jobs and individuals employed in nj according to ui records
df_wages %>%
    summarize(
        n_ind = n_distinct(hashed_ssn),
        n_jobs = n()
    )

For reference, you can compare the number of individuals who were employed in at least one quarter according to the wage records within their first 13 quarters after graduation with the total number of individuals in the original cohort.

In [ ]:
df_cohort %>%
    summarize(
        n_ind = n_distinct(hashed_ssn)
    )

Does this proportion surprise you?

Additionally, we assume that there should be one entry for each individual-employer-quarter combination. Let's confirm that assumption by counting the number of entries within each `hashed_ssn`-`hashed_ein`-`yrq` combination.

In [ ]:
# see if people have multiple filings for same employer and year quarter
df_wages %>%
    group_by(hashed_ssn, hashed_ein, yrq) %>%
    count() %>%
    arrange(desc(n)) %>%
    head()

It turns out that it is possible to have more than one observation within a `hashed_ssn`-`hashed_ein`-`yrq` combination. This occurs because employers are allowed to re-file to correct previously submitted employment records. To only keep the most recent records, we select the most recent `filedate` within this combination.

In [ ]:
# unduplicate wages
df_wages_undup <- df_wages %>%
    arrange(hashed_ssn, hashed_ein, yrq, desc(filedate)) %>%
    distinct(hashed_ssn, hashed_ein, yrq, .keep_all=TRUE)

In [ ]:
# see number of jobs and individuals employed in nj according to ui records
df_wages_undup %>%
    summarize(
        n_ind = n_distinct(hashed_ssn),
        n_jobs = n()
    )

For reference, by comparing the `n_jobs` within the code cell above and the one for `df_wages`, you can see that relatively few records contained duplicate entries differing by `filedate`. However, it is important to make sure information is not being potentially duplicated or miscast. Let's confirm that we successfully unduplicated the `df_wages_undup`.

In [ ]:
# see that people can have multiple filings for same employer and year quarter
df_wages_undup %>%
    group_by(hashed_ssn, hashed_ein, yrq) %>%
    count() %>%
    arrange(desc(n)) %>%
    head()

`df_wages_undup` is saved as a table in the `tr_nj_2021` database as `nb_cohort_wages_link`. The code to create `nb_cohort_wages_link` is available in the supplemental notebook.

## 7. Employment Outcomes

Connecting education data to employment data is only the first part of understanding the outcomes for New Jersey graduates. There are many ways that one could define and evaluate these outcomes. We present a few here and several more in the Appendix. While working through this section, think through what outcomes would be most relevant for your research question.

We will look at the following outcomes:
- Average quarterly earnings and the total workers employed by quarter (data: all wages)
    - Distribution by major
- Average quarterly earnings and the total workers employed by quarter (data: dominant wages)
    - Distribution by major
- Full quarter employment (data: full quarter wages, non-dominant)
    - Distribution by sex
- Employment patterns (data: dominant wages)

### Average quarterly earnings and number employed by quarter

As mentioned earlier, we plan to focus on the first 12 quarters post-graduation for each individual. However, `df_wages_undup` currently contains employment outcomes in the quarter of graduation, as well as 13 quarters post-graduation. To isolate each quarter post-graduation, we will create a new variable, `quarter_number`, which uses some of R's functionality when it comes to working with date variables. After converting `grad_date` and `job_date` to date objects in R, we can calculate the difference in weeks between the two values (thanks to `difftime`) by dividing by 13 since there are roughly 13 weeks in each quarter and rounding to the nearest whole number.

In [ ]:
# get quarter from graduation
df_wages_undup <- df_wages_undup %>%
    mutate(
        quarter_number = round(as.double(difftime(as.Date(job_date), as.Date(grad_date), units = "weeks")/13), 0)
    )

# see evidence
df_wages_undup %>%
    select(grad_date, job_date, quarter_number) %>%
    head()

With `quarter_number`, we can sum each individual's total earnings by quarter, excluding all observations where `quarter_number` is either 0 or 13.

In [ ]:
# ignore quarters 0 and 13
df_wages_undup <- df_wages_undup %>%
    filter(!(quarter_number %in% c(0, 13)))

# find quarterly wages
df_wages_undup %>%
    group_by(hashed_ssn, quarter_number) %>%
    summarize(
        quarterly_wages = sum(wage),
    ) %>%
    ungroup() %>%
    head()

Let's save these results to the data frame `quarterly_wages`, so we can compute the cohort average quarterly earnings.

In [ ]:
# save as quarterly_wages
quarterly_wages <- df_wages_undup %>%
    group_by(hashed_ssn, quarter_number) %>%
    summarize(
        total_wages = sum(wage),
    ) %>%
    ungroup()

Now that we have `quarterly_wages` to capture each individual's total quarterly earnings, we can compute the cohort's average quarterly earnings and the total number of individuals employed, broken down by quarter after graduation.

In [ ]:
# average wages and number of grads with wages by quarter after graduation
avg_and_num <- quarterly_wages %>%
    group_by(quarter_number) %>%
    summarize(
        mean_wage = mean(total_wages),
        n_employed = n_distinct(hashed_ssn)
    )

avg_and_num

We can see that after about REDACTED quarters post-graduation, the number of individuals employed in New Jersey in our cohort begins to REDACTED, but that over time, the average quarterly earnings REDACTED. Keep in mind that the `mean_wage` encompasses average quarterly earnings, so if an individual had multiple earning sources in a quarter, we are currently taking the sum of them. Let's see if we see similar trends amongst those receiving the most common degrees within the cohort.

#### By Major

Recall the first data exploration notebook, where we identified the most common degrees earned within the cohort. We used the following code to isolate the most common majors:

    df_common_major <- df %>%
        count(cip_family) %>%
        arrange(desc(n)) %>%
        mutate(
            prop = n/sum(n)
        ) %>%
        head(10)
        
Let's see if there are consistent trends relative to the entire cohort of earners for those who received degrees in the two most common fields. Recall that whereas the cohort was saved in the data frame `df` in the first notebook, it is saved as `df_cohort` here.

In [ ]:
# 2 most common majors
com_majors <- df_cohort %>%
    count(cip_family, cip2) %>%
    arrange(desc(n)) %>%
    mutate(
        prop = n/sum(n)
    ) %>%
    head(2)

com_majors

Now that we have the two most common majors, we can match this to `df_wages_undup` and evaluate earnings and the number of individuals employed within these two majors.

In [ ]:
# earnings and number employed for most common majors
avg_and_num_major <- df_wages_undup %>%
    group_by(hashed_ssn, quarter_number, major) %>%
    summarize(
        total_wages = sum(wage)
    ) %>%
    ungroup() %>%
    filter(substring(major, 1,2) %in% com_majors$cip2) %>%
    group_by(substring(major, 1, 2), quarter_number) %>%
    summarize(
        mean_wage = mean(total_wages),
        n_employed = n_distinct(hashed_ssn)
    )

avg_and_num_major

Naturally, we may wonder if we see the same trend amongst `mean_wage` when isolating for dominant earnings within a quarter, the job where the individual had the highest wages per quarter.

### Average quarterly earnings and number employed by quarter (Dominant Earnings)

Before we can perform our analysis, we need to restrict `df_wages_undup` to only include the highest earnings per quarter for each individual. We will take a similar approach to when we unduplicated wage records, by first arranging employment for each individual/quarter combination by descending wages before taking the highest earnings. We will save this resulting data frame as `df_dom_wages`.

In [ ]:
# identify dominant wages in each quarter
df_dom_wages <- df_wages_undup %>%
    arrange(hashed_ssn,yrq, desc(wage)) %>%
    distinct(hashed_ssn, yrq, .keep_all=TRUE)

head(df_dom_wages)

In [ ]:
# confirm we have one entry for each hashed_ssn/yrq combination
df_dom_wages %>%
    count(hashed_ssn, yrq) %>%
    arrange(desc(n)) %>%
    head()

Now that we have isolated dominant earnings, we can recycle the same code that we applied to `df_wages_undup`.

In [ ]:
# average wages and number of grads with wages by quarter after graduation
avg_and_num_dom <- df_dom_wages %>%
    group_by(quarter_number) %>%
    summarize(
        mean_wage = mean(wage),
        n_employed = n_distinct(hashed_ssn)
    )

avg_and_num_dom

Interestingly enough, you can see that while a similar trend appears within `mean_wage`, it does not REDACTED for every REDACTED. Let's see if this trend is similar for graduates of the two most common majors.

#### By Major

In [ ]:
# average wages and number of grads with wages by quarter after graduation for common majors
avg_and_num_dom_major <- df_dom_wages %>%
    group_by(hashed_ssn, quarter_number, major) %>%
    summarize(
        total_wages = sum(wage)
    ) %>%
    ungroup() %>%
    filter(substring(major, 1,2) %in% com_majors$cip2) %>%
    group_by(substring(major, 1, 2), quarter_number) %>%
    summarize(
        mean_wage = mean(total_wages),
        n_employed = n_distinct(hashed_ssn)
    )

avg_and_num_dom_major

### Full-Quarter Employment
Interestingly enough, you can see different REDACTED growth experiences, on average, between graduates of these two majors. However, these two employment measures, using total quarterly and dominant wages, do not include any measure of employment stability, which is often an important job aspect, especially for recent graduates.
    
There are many ways in which to define stable employment. Sometimes, it may be useful to define stable employment by a consecutive number of quarters worked with the same employer. Other times, stable employment may assume an alternative definition. Here, we will define stable employment as REDACTED employment. REDACTED employment in quarter *t* is indicated by a presence of wages with the same employer in quarters *t-1*, *t*, and *t+1*.

**Example: Data Needed to Assess Full-Quarter (FQ) Employment for Person 1/Employer A in Each of the 4 Quarters Post-Graduation**

|Person/Employer Combination|High Degree |FQ YearQtr |t-1|t|t+1|
|---|---|---|---|---|---|
|_Person 1/Employer A_ |_2013 Q3_ |_2013 Q4_ |<font color=green>2013 Q3</font> |**2013 Q4** |2014 Q1 |
|_Person 1/Employer A_ |_2013 Q3_ |_2014 Q1_ |2013 Q4 |**2014 Q1** |2014 Q2 |
|_Person 1/Employer A_ |_2013 Q3_ |_2014 Q2_ |2014 Q1 |**2014 Q2** |2014 Q3 |
|_Person 1/Employer A_ |_2013 Q3_ |_2014 Q3_ |2014 Q2 |**2014 Q3** |<font color=green>2014 Q4</font> |

As can be seen in the table above, calculating REDACTED employment for the same four quarter span requires REDACTED additional quarters of wage information. This requires us to extend our data frame to include the employment quarter of graduation as well as one additional quarter after our final quarter of interest. In this example, it means including employment in the quarter prior to quarter REDACTED and the quarter after quarter REDACTED. Now, it should be clear as to why we initially brought in REDACTED quarters worth of data, as we need all REDACTED quarters to calculate REDACTED employment within the REDACTED on which we want to focus.

In practice, to isolate instances of REDACTED employment, we can do so by creating three copies of our `nb_cohort_wages_link` table, and matching them based on `hashed_ein` and `hashed_ssn` values while accounting for `job_date` differences amounting to quarters t-1, t, and t+1.

In [ ]:
# get full quarter instances
qry <- "
select b.hashed_ssn, b.hashed_ein, b.wage, b.job_date, b.grad_date, b.sex, b.major
from tr_nj_2021.dbo.nb_cohort_wages_link a, tr_nj_2021.dbo.nb_cohort_wages_link b, tr_nj_2021.dbo.nb_cohort_wages_link c
where a.hashed_ssn = b.hashed_ssn and a.hashed_ein = b.hashed_ein and a.job_date = dateadd(month, 3, b.job_date)
and a.hashed_ssn = c.hashed_ssn and a.hashed_ein = c.hashed_ein and b.job_date = dateadd(month, 3, c.job_date)
"
full_q_wages <- dbGetQuery(con, qry)

head(full_q_wages)

Let's check how many individuals experienced at least one quarter of stable employment, as well as the number of employers by which members of the cohort experienced stable employment and their average wages.

In [ ]:
# see number with at least one quarter of full quarter, what are their average wages
full_q_stats <- full_q_wages %>%
    summarize(
        num_individuals = n_distinct(hashed_ssn),
        num_employers = n_distinct(hashed_ein),
        avg_wage = mean(wage)
    )

full_q_stats

Is there a difference by sex? In our next subsection, we will try to answer this question.

#### By Sex

At the end of the first data exploration notebook, after exploring the most common majors within the cohort, we also analyzed the sex breakdown of the cohort. Here, we will compare the overall sex breakdown to that of those who experienced at least one quarter of full-quarter employment.

Recall the code from the first notebook in the code cell below:      

In [ ]:
# sex breakdown
df_sex <- df_cohort %>%
    count(sex) %>%
    arrange(desc(n)) %>%
    mutate(
        prop = n/sum(n)
    )

# see df_sex
df_sex

In [ ]:
# see number with at least one quarter of full quarter, what are their average wages
full_q_stats_sex <- full_q_wages %>%
    group_by(sex) %>%
    summarize(
        num_individuals = n_distinct(hashed_ssn),
        num_employers = n_distinct(hashed_ein),
        avg_wage = mean(wage)
    ) %>%
    mutate(
        prop = num_individuals/sum(num_individuals)
    )

full_q_stats_sex

We can see that roughly the REDACTED proportion exists, and interestingly enough, of those who experienced stable employment, those of REDACTED experienced REDACTED wages, on average.

------

#### **Checkpoint 3: Stable Employment by Major**

Recreate the table above (besides `prop`) for the two most commmon majors. Do the results surprise you?

In [ ]:
# replace ___
full_q_wages %>%
    filter(substring(___, 1, 2) %in% com_majors$cip2) %>%
    group_by(___) %>%
    summarize(
        num_individuals = n_distinct(hashed_ssn),
        num_employers = n_distinct(hashed_ein),
        avg_wage = mean(wage)
    )

------

### Employment Patterns

At the end of this section, we hope to have found the most common employment patterns for everyone in the original cohort, not just those who matched to the UI wage records. To start, let's get a sense of the amount of individuals that are missing from `df_dom_wages`.

In [ ]:
# see size of original cohort
df_cohort %>%
    summarize(
        num_inds = n_distinct(hashed_ssn)
    )

In [ ]:
# see amount of people with employment outcomes
df_dom_wages %>%
    summarize(n_distinct(hashed_ssn))

Before we manipulate any existing data frames, let's confirm that if we join `df_dom_wages` to `df_cohort`, the number of individuals where `quarter_number` (or pick any other variable unique to `df_dom_wages`) is equal to the difference in individuals between `df_cohort` and `df_dom_wages`.

In [ ]:
# see that everyone with na quarter is equal to amount who didn't show up in df_dom_wages
df_cohort %>%
    left_join(df_dom_wages, by = "hashed_ssn") %>%
    filter(is.na(quarter_number)) %>%
    summarize(n_distinct(hashed_ssn))

Now that we have confirmed that our join should work as intended, as there are no instances of any observations that may be duplicated, we will join `df_dom_wages` to `df_cohort`. After doing so, we will set all instances where `quarter_number` is `NA` equal to 1, so that we will eventually be able to have REDACTED observations for each individual, one for each potential quarter of employment.

In [ ]:
# set all where quarter is na equal to one so we can use complete
full_wages <- df_cohort %>%
    left_join(df_dom_wages, by = c("hashed_ssn", "grad_date")) %>%
    mutate(
        quarter_number = ifelse(is.na(quarter_number), 1, quarter_number)
    )

# see potential quarter numbers
full_wages %>%
    distinct(quarter_number) %>%
    arrange(quarter_number)

Now that we have all potential `hashed_ssn` values, as well as instances of all desired `quarter_number` values, we can leverage `complete`, which will add additional rows for any combinations of `hashed_ssn`/`quarter_number` that do not currently exist in `full_wages`. If the combination does not appear in `full_wages`, the resulting `wage` value will be `NA`, signifying the individual was not employed in this quarter. As verification, the number of rows should equal the number of individuals multiplied by REDACTED.

In [ ]:
# complete file
completed <- full_wages %>%
    complete(hashed_ssn, quarter_number, fill=list(wage=NA))

# see than n should be a multiple of n_dist
completed %>%
    summarize(
        n = n(),
        n_inds = n_distinct(hashed_ssn)
    )

In [ ]:
# see completed
head(completed)

Now that we have created `completed`, we just need to aggregate and manipulate the data frame so that each column is a quarter, and each observation is an individual, with the corresponding columns indicating whether the individual was employed in the given quarter. To start, let's create a variable `wage_ind`, which will be "yes" if the individual was employed in the quarter, and "no" otherwise. Additionally, for each in column manipulation, we will change each `quarter_number` value from 1, 2, 3,..., 12 to Q1, Q2, Q3,..., Q12 and call this variable `quarter`.

In [ ]:
# create wage_ind and quarter variables
patterns <- completed %>%
    mutate(
        wage_ind = ifelse(is.na(wage), "no", "yes"),
        quarter = paste("Q",quarter_number, sep="")
    )

head(patterns)

Now, we need to figure out how to "pivot" the data frame so that each column is a value of `quarter`, with `wage_ind` values for the `hashed_ssn` values. To do so, we will use `pivot_wider`, which allows us to take a tidy data frame (one observation per row) and "widen" it so that each column becomes values from what was previously a single column (`quarter`) and the rows are occupied by those from a corresponding column (`wage_ind`). 

After manipulating the data frame, we can aggregate by the quarter columns, and count the number of observations within each of these patterns to discover the most common employment patterns for the cohort.

In [ ]:
# find most common employment patterns
patterns <- patterns %>%
    select(hashed_ssn, quarter, wage_ind) %>%
    pivot_wider(names_from = quarter, values_from = wage_ind) %>%
    group_by(Q1, Q2, Q3, Q4, Q5, Q6, Q7, Q8, Q9, Q10, Q11, Q12) %>%
    summarize(cnt = n_distinct(hashed_ssn)) %>%
    arrange(desc(cnt)) %>%
    ungroup() %>%
    mutate(
        prop = percent(cnt/sum(cnt), .01)
    ) 

head(patterns)

## 8. Save as csvs

Before we finish the notebook, let's save your work as .csv files so that they can be referenced in the Data Visualization notebook.

<font color=red> Note that you need to change the directory in write.csv() statements below. Replace ". ." with your username.</font>

In [ ]:
# Save dataframes to CSV to use in later notebook

# average quarterly earnings and number employed by quarter
write_csv(avg_and_num, "U:\\..\\NJ Training\\Results\\avg_and_num.csv")

# average quarterly earnings and number employed by quarter (common majors)
write_csv(avg_and_num_major, "U:\\..\\NJ Training\\Results\\avg_and_num_major.csv")

# average dominant quarterly earnings and number employed by quarter
write_csv(avg_and_num_dom, "U:\\..\\NJ Training\\Results\\avg_and_num_dom.csv")

# average dominant quarterly earnings and number employed by quarter (common majors)
write_csv(avg_and_num_dom_major, "U:\\..\\NJ Training\\Results\\avg_and_num_dom_major.csv")

# full quarter info
write_csv(full_q_stats, "U:\\..\\NJ Training\\Results\\full_q_stats.csv")

# full quarter info by sex
write_csv(full_q_stats_sex, "U:\\..\\NJ Training\\Results\\full_q_stats_sex.csv")

# employment patterns
write_csv(patterns, "U:\\..\\NJ Training\\Results\\patterns.csv")

## **Appendix**

Here we provide code to look at additional employment outcomes and to develop industry codes. The appendix provides code for the following:
- At least four consecutive quarters of full quarter employment (data: full quarter examples, non-dominant)
- Wage progression within full quarter employment (data: full quarter wages, dominant)
- Industry measures

### Stable Employment: 4 Consecutive quarters of Full-Quarter Employment
When introducing full-quarter employment as a proxy for stable employment, we noted that in other situations, stable employment can be defined by a consecutive amount of quarters. In this next example, we will combine these two concepts, defining stable employment as the existence of full-quarter employment across four consecutive quarters, all with the same employer. Here, we will not isolate dominant wages, but rather consider all stable employment instances in this measure.

At this point, we have already identified all instances of full-quarter employment and saved the results to `full_q_wages`. Now, we have to track consecutive quarters, or streaks, with employment for the same `hashed_ssn` and `hashed_ein`. To do so, we will first structure the data frame so that `job_date` values are arranged by `hashed_ssn`/ `hashed_ein` combinations. Then, after establishing a baseline of the first date of employment for each `hashed_ssn`/ `hashed_ein` combination, we can calculate the difference in quarters elapsed between the first instance and each future observation. Finally, we consider a streak to "continue" if the time elapsed (`interval`) is one quarter greater than the time elapsed of the previous observation (`lag(interval) + 1`) and count consecutive instances where we have consecutive quarters for each `hashed_ein`/`hashed_ssn` combination (`sequence(rle(consecutive)$lengths)`.

In [ ]:
# track streaks within each employer/individual combination
track_streaks <- full_q_wages %>%
    group_by(hashed_ssn, hashed_ein) %>%
    arrange(hashed_ssn, hashed_ein, job_date) %>%
    mutate(
        first_job = min(job_date),
        interval = round(as.double(difftime(as.Date(job_date), as.Date(first_job), units = "weeks")/13), 0),
        consecutive = interval == lag(interval) + 1,
        consecutive = ifelse(is.na(consecutive), lead(consecutive), consecutive)
    ) %>%
  mutate(
    Streak = ifelse(consecutive == TRUE, sequence(rle(consecutive)$lengths), 0)
  ) %>%
    ungroup()

head(track_streaks)

From here, we can `filter` `track_streaks` for any instances where `Streak` is 4.

In [ ]:
# see how many had a streak of at least 4
track_streaks %>%
    filter(Streak == 4) %>%
    summarize(
        n_distinct(hashed_ssn)
    )


We started this section by analyzing wage growth and the number of individuals employed by quarter after graduation. The next example will help you get a better sense of wage progression for those experiencing full-quarter employment.

### Full-quarter Employment Wage Growth and Time Elapsed

Given the isolation of those experiencing full-quarter employment (`full_q_wages`), we can further our understanding of stable employment experiences by comparing wage progression within this subset. To do so, for each person, we will first identify dominant full-quarter wages within each quarter, taking the highest wages per quarter of full-quarter employment for each individual. After doing so, we will compare earnings between the first instance of full-quarter employment and the last, while also tracking the average time elapsed between these occurrences. 

In [ ]:
# identify dominant full quarter wages in each quarter
full_q_wages_dom <- full_q_wages %>%
    arrange(hashed_ssn,job_date, desc(wage)) %>%
    distinct(hashed_ssn, job_date, .keep_all=TRUE)

head(full_q_wages_dom)

Now that we have identified only the dominant full-quarter experiences for each individual, we will track wage growth and time elapsed by first creating two data frames, `first_full` and `last_full`, which track the first and last full-quarter employment experiences per `hashed_ssn`. Afterwards, we can combine these two data frames to create `full_history`, and then perform a simple subtraction between these columns.

In [ ]:
# get first quarter of earnings
first_full <- full_q_wages_dom %>%
    arrange(hashed_ssn, job_date) %>%
    distinct(hashed_ssn, .keep_all = TRUE) %>%
    rename(
        job_date_first = job_date,
        wage_first = wage
    )

# get last quarter of earnings
last_full <- full_q_wages_dom %>%
    arrange(hashed_ssn, desc(job_date)) %>%
    distinct(hashed_ssn, .keep_all = TRUE) %>%
    rename(
        job_date_last = job_date,
        wage_last = wage
    ) %>%
    select(-c(hashed_ssn, hashed_ein, grad_date, sex))

full_history <- cbind(first_full, last_full)

head(full_history)

In [ ]:
# see wage growth and time elapsed between first and last full quarter jobs in this time frame
full_history %>% 
    mutate(
        salary_growth = wage_last - wage_first,
        quarter_diff = round(as.double(difftime(as.Date(job_date_last), as.Date(job_date_first), units = "weeks")/13), 0)
    ) %>%
    summarize(
        mean_salary_growth = mean(salary_growth),
        mean_quarter_diff = mean(quarter_diff)
    )

As you can see there appears to be a REDACTED average salary REDACTED, as well as a REDACTED REDACTED between the REDACTED and REDACTED instances of REDACTED employment.

Now that we have demonstrated how to implement various measures of employment outcomes in R, you should have a better sense of the employment outcomes of the cohort. Of course, this is not an exhaustive list of examples, and you are encouraged to implement employment outcome measures that make most sense given your specific research question. For the last section of the wage exploration, we will try to understand common employment patterns over the course of the 12 quarters post-graduation.

### Developing industry measures

Although important, working with industries introduces a separate level of complexity. The `ui_employer` table in the `ds_nj_dol` database contains REDACTED information by `hashed_fein` for the most recent quarter of wage records. Let's start by reading this data at the two-digit level into R as `df_employer`.

In [ ]:
# take employers, first two digits of naics
# include where ind_active = 'Y' so only include active industries
qry <- "
SELECT hashed_fein, substring(naics, 1, 2) as naics
FROM ds_nj_dol.dbo.ui_employer
where substring(naics, 1, 2) is not null and substring(naics, 1, 2) not in ('00', '99') and ind_active = 'Y'
"

df_employer <- dbGetQuery(con,qry)

head(df_employer)

Ideally, each employer would be associated with one industry, or NAICS code. However, you can probably think of an employer that provides services in multiple industries, even when using two-digit NAICS codes. Let's see if there are any `hashed_fein` values with multiple NAICS codes.

In [ ]:
# see max amount of unique naics for each employer
df_employer %>%
    group_by(hashed_fein) %>%
    summarize(
        num_naics = n_distinct(naics)
    ) %>%
    arrange(desc(num_naics)) %>%
    head()

There are certain rules and decisions you may be able to make to choose between NAICS codes for employers, and most employers do not have multiple NAICS codes. However, we will leave it to your group to discuss potential ways to de-duplicate `hashed_fein` values. You can find the most common industries of dominant employers using the following code, where we join `df_employer` to `df_dom_wages` before finding the three most common industries.

> Note: There was no industry de-duplication applied in this code.

In [ ]:
# find most common industries
common_inds <- df_dom_wages %>%
    left_join(df_employer, by = c("hashed_ein" = "hashed_fein")) %>%
    filter(!is.na(naics), naics != "99") %>%
    group_by(naics) %>%
    summarize(
       n = n_distinct(hashed_ssn)
    ) %>%
    arrange(desc(n)) %>%
    head(3)

common_inds

We can find the industries associated with these NAICS codes using the `naics_descriptions` table in the `ds_public_1` database. 

In [ ]:
# read in naics descriptions
qry <- "
select Code, Title 
from ds_public_1.dbo.naics_descriptions 
"
naics_groups <- dbGetQuery(con, qry)

head(naics_groups)

In [ ]:
# get naics descriptions
common_inds_desc <- common_inds %>%
    left_join(naics_groups, c("naics" = "Code"))

head(common_inds_desc)